In [1]:
import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

def process(X,file):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset

def augmentation(row,label,Augmented,Y_Augmeneted):
    scale_out = skimage.transform.rescale(row, scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(100,100))
    scale_out2 = skimage.transform.rescale(row, scale=1.5, mode='constant')
    scale_out2=skimage.transform.resize(scale_out2,(100,100))
    scale_in= skimage.transform.rescale(row, scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(100,100))
    scale_in2= skimage.transform.rescale(row, scale=0.8, mode='constant')
    scale_in2=skimage.transform.resize(scale_in2,(100,100))
    rot = skimage.transform.rotate(row, angle=20, mode='reflect')
    rot=skimage.transform.resize(rot,(100,100))
    images=[row,scale_out,scale_in,scale_out2,scale_in2,rot]
    for image in images:
        Augmented.append(image)
        Y_Augmented.append(label)

In [4]:
import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

Normal=[]
Bacterial=[]
Virus=[]
classes=[Virus,Bacterial,Normal]
Y_Normal=[]
Y_Bacterial=[]
Y_Virus=[]
Y=[Y_Normal,Y_Bacterial,Y_Virus]
files=glob.glob('chest_xray_lower_dim2'+'/*.jpeg')
ra.shuffle(files)
for file in files:
    k=-1
    if "virus" in file:
        k=0
    elif "bacteria" in file:
        k=1
    else:
        k=2
    Y[k].append(str(k))
    process(classes[k],file)
    
len_classes=[]
for class_ in classes:
    len_classes.append(len(class_))
print(len_classes)
same_length=min(len_classes)
evenly=False

if evenly:
    for i in range(3):
        len_classes[i]=same_length

for i in range(3):
    classes[i]=classes[i][0:len_classes[i]]
for i in range(3):
    Y[i]=Y[i][0:len_classes[i]]

Train=[]
Valid=[]
Test=[]
TVT=[Train,Valid,Test]
Y_Train=[]
Y_Valid=[]
Y_Test=[]
Y_TVT=[Y_Train,Y_Valid,Y_Test]

bounds=[0,0.64,0.8,1]
for i in range(3):
    for j in range(3):
        TVT[i]+=classes[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        Y_TVT[i]+=Y[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        
Augmented=[]
Y_Augmented=[]

for i in range(len(Train)):
    augmentation(Train[i],Y_Train[i],Augmented,Y_Augmented)

print(len(Train))

TVT[0]=Augmented
Y_TVT[0]=Y_Augmented

for i in range(3):
    TVT[i]=normalize(TVT[i])
    TVT[i]=TVT[i].reshape(-1,100,100,1)
    Y_TVT[i]=keras.utils.to_categorical(Y_TVT[i],3)

print(len(Augmented))

[1493, 2780, 1583]
3747
22482


In [5]:
# epoch=32 steps_per_epoch=32

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_layers,neur_layers):
    
    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu')(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax')(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,mode='min')
    model=mlp_model()
    model.fit(TVT[0],Y_TVT[0],validation_data=(TVT[1],Y_TVT[1]),epochs=epochs_num,batch_size=batch_len,callbacks=[callback],verbose=1)
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(X_test, Y_test,verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,300,3,3)

Epoch 1/100
75/75 [==============================] - 172s 2s/step - loss: 1.0598 - accuracy: 0.4748 - val_loss: 1.0237 - val_accuracy: 0.4749
Epoch 2/100
75/75 [==============================] - 164s 2s/step - loss: 1.0141 - accuracy: 0.4995 - val_loss: 0.7763 - val_accuracy: 0.6521
Epoch 3/100
75/75 [==============================] - 162s 2s/step - loss: 0.9019 - accuracy: 0.5679 - val_loss: 0.7592 - val_accuracy: 0.7065
Epoch 4/100
75/75 [==============================] - 170s 2s/step - loss: 0.6663 - accuracy: 0.6959 - val_loss: 0.6614 - val_accuracy: 0.7054
Epoch 5/100
75/75 [==============================] - 159s 2s/step - loss: 0.6339 - accuracy: 0.7174 - val_loss: 0.6475 - val_accuracy: 0.7065
Epoch 6/100
75/75 [==============================] - 211s 3s/step - loss: 0.6216 - accuracy: 0.7228 - val_loss: 0.6272 - val_accuracy: 0.7225
Epoch 7/100
75/75 [==============================] - 202s 3s/step - loss: 0.5921 - accuracy: 0.7427 - val_loss: 0.6149 - val_accuracy: 0.7236
Epoch 

NameError: name 'X_test' is not defined

#### Let's add alpha 0.0001  3 augmentations and batch 150

In [7]:
import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

def process(X,file):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset

def augmentation(row,label,Augmented,Y_Augmeneted):
    scale_out = skimage.transform.rescale(row, scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(100,100))
    scale_in= skimage.transform.rescale(row, scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(100,100))
    rot = skimage.transform.rotate(row, angle=20, mode='reflect')
    rot=skimage.transform.resize(rot,(100,100))
    images=[row,scale_out,scale_in,rot]
    for image in images:
        Augmented.append(image)
        Y_Augmented.append(label)



import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

Normal=[]
Bacterial=[]
Virus=[]
classes=[Virus,Bacterial,Normal]
Y_Normal=[]
Y_Bacterial=[]
Y_Virus=[]
Y=[Y_Normal,Y_Bacterial,Y_Virus]
files=glob.glob('chest_xray_lower_dim2'+'/*.jpeg')
ra.shuffle(files)
for file in files:
    k=-1
    if "virus" in file:
        k=0
    elif "bacteria" in file:
        k=1
    else:
        k=2
    Y[k].append(str(k))
    process(classes[k],file)
    
len_classes=[]
for class_ in classes:
    len_classes.append(len(class_))
print(len_classes)
same_length=min(len_classes)
evenly=False

if evenly:
    for i in range(3):
        len_classes[i]=same_length

for i in range(3):
    classes[i]=classes[i][0:len_classes[i]]
for i in range(3):
    Y[i]=Y[i][0:len_classes[i]]

Train=[]
Valid=[]
Test=[]
TVT=[Train,Valid,Test]
Y_Train=[]
Y_Valid=[]
Y_Test=[]
Y_TVT=[Y_Train,Y_Valid,Y_Test]

bounds=[0,0.64,0.8,1]
for i in range(3):
    for j in range(3):
        TVT[i]+=classes[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        Y_TVT[i]+=Y[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        
Augmented=[]
Y_Augmented=[]

for i in range(len(Train)):
    augmentation(Train[i],Y_Train[i],Augmented,Y_Augmented)

print(len(Train))

TVT[0]=Augmented
Y_TVT[0]=Y_Augmented

for i in range(3):
    TVT[i]=normalize(TVT[i])
    TVT[i]=TVT[i].reshape(-1,100,100,1)
    Y_TVT[i]=keras.utils.to_categorical(Y_TVT[i],3)

print(len(Augmented))

# epoch=32 steps_per_epoch=32

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_layers,neur_layers):
    
    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for i in range(conv_layers):
            hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Conv2D(40,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for i in range(neur_layers):
            i2=i+1
            hidden=layers.Dense(30,activation='relu',kernel_regularizer=regularizers.L2(0.0001))(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        nl=neur_layers+1
        output=layers.Dense(3,activation='softmax',kernel_regularizer=regularizers.L2(0.0001))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4,mode='max')
    model=mlp_model()
    model.fit(TVT[0],Y_TVT[0],validation_data=(TVT[1],Y_TVT[1]),epochs=epochs_num,batch_size=batch_len,callbacks=[callback],verbose=1)
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(TVT[2], Y_TVT[2],verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,150,3,3)

[1493, 2780, 1583]
3747
14988
Epoch 1/100
100/100 [==============================] - 150s 1s/step - loss: 1.0683 - accuracy: 0.4714 - val_loss: 1.0502 - val_accuracy: 0.4749
Epoch 2/100
100/100 [==============================] - 121s 1s/step - loss: 1.0146 - accuracy: 0.5052 - val_loss: 0.7321 - val_accuracy: 0.6862
Epoch 3/100
100/100 [==============================] - 119s 1s/step - loss: 0.9378 - accuracy: 0.5441 - val_loss: 0.6482 - val_accuracy: 0.6969
Epoch 4/100
100/100 [==============================] - 123s 1s/step - loss: 0.7038 - accuracy: 0.6856 - val_loss: 0.6277 - val_accuracy: 0.7086
Epoch 5/100
100/100 [==============================] - 129s 1s/step - loss: 0.6521 - accuracy: 0.7091 - val_loss: 0.5843 - val_accuracy: 0.7321
Epoch 6/100
100/100 [==============================] - 119s 1s/step - loss: 0.6241 - accuracy: 0.7269 - val_loss: 0.5614 - val_accuracy: 0.7556
Epoch 7/100
100/100 [==============================] - 119s 1s/step - loss: 0.5971 - accuracy: 0.7489 - va